In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementNotVisibleException, ElementNotSelectableException
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from unidecode import unidecode
from urllib.parse import urlparse, parse_qs
from datetime import datetime

In [2]:
driver = webdriver.Chrome()

In [20]:
driver.get("https://www.maeva.com/fr-fr/residence-pierre---vacances-les-combes_10790.html")

In [21]:
soupe = BeautifulSoup(driver.page_source)

In [31]:
months = {
    'jan':'01',
    'fev':'02',
    'mars':'03',
    'avr':'04',
    'mai':'05',
    'juin':'06',
    'juil':'07',
    'aout':'08',
    'sept':'09',
    'oct':'10',
    'nov':'11',
    'dec':'12'
}

In [22]:
toasts = soupe.find('div', {'id':'fiche-seo-toaster-anchor'}).find_all(
            'div', {'class':'toaster fiche-bloc-produit-item-b cursor-auto w100'})

In [23]:
len(toasts)

3

In [24]:
toasts[0].find('span', {'class':'price_item_final'}).find('span').text.strip()[:-1]

'332'

In [25]:
def extract():

    def link_params(url):
        url_params = parse_qs(urlparse(url).query)
        sep = '/'
        try:
            n_offre = ''.join(url_params['id'][0].split('-')[::-1])
            start_date = sep.join(
                url_params['date_debut'][0].split('-')[::-1])
            end_date = sep.join(url_params['date_fin'][0].split('-')[::-1])
            return n_offre, start_date, end_date
        except KeyError as e:
            print(e)
            return

    data = []

    soupe = BeautifulSoup(driver.page_source, 'lxml')

    if soupe.find('div', class_='fiche-seo-toaster-container'):
        toasters = soupe.find('div', {'id':'fiche-seo-toaster-anchor'}).find_all('div', {'class':'toaster fiche-bloc-produit-item-b cursor-auto w100'}) \
        if soupe.find('div', {'id':'fiche-seo-toaster-anchor'}).find_all('div', {'class':'toaster fiche-bloc-produit-item-b cursor-auto w100'}) else []
        residence = soupe.find('h1', {"id": "fiche-produit-residence-libelle"}).text.strip() \
            if soupe.find('h1', {"id": "fiche-produit-residence-libelle"}) else ''
        localisation = soupe.find('button', {"id": "fiche-produit-localisation"}).find('span', class_='fs-5').text.strip() \
            if soupe.find('button', {"id": "fiche-produit-localisation"}) else ''

        breadcrumbs = []
        
        try:
            breadcrumbs = soupe.find(
                'ol', {'id': 'ui-ariane'}).find_all('li', {'itemprop': 'itemListElement'})
        except:
            breadcrumbs = soupe.find(
                'nav', {'id': 'ui-ariane'}).find_all('div', {'itemprop': 'itemListElement'})

        # station_breadcrumb = breadcrumbs[-2:-1][0].find('a', class_='ariane-item') if breadcrumbs[-2:-1][0].find('a', class_='ariane-item') else ''
        # station_name = localisation
        # station_key = station_breadcrumb['href'].split(',')[1].replace('.html', '') if station_breadcrumb != '' else ''

        # if not station_key:
        #     station_key = self.stations[station_name.upper()] if station_name.upper() in self.stations.keys() else ''
        #     print(station_name, ' => ', station_key)

        for toaster in toasters:
            is_disponible = False if toaster.find(
                'div', {'id': 'toaster-right-date'}).find('div', class_='font-logement-disabled') else True

            if is_disponible:
                dat = {}

                typologie = toaster.find('div', class_="toaster-residence-libelle-container").text.strip() \
                    if toaster.find('div', class_='toaster-residence-libelle-container') else ''
                prix_actuel = toaster.find('span', {'class':'price_item_final'}).find('span').text.strip()[:-1] \
                     if toaster.find('span', {'class':'price_item_final'}).find('span') else 0.00
                prix_init = toaster.find('span', {'class':'price_item_barre'}).text.strip()[:-1] \
                    if toaster.find('span', {'class':'price_item_barre'}) else prix_actuel
                link = 'www.maeva.com' + toaster.find("div", class_="toaster-right-cta").find("a", href=True)['href'] \
                    if toaster.find("div", class_="toaster-right-cta") else ''

                n_offres, date_debut, date_fin = link_params(link)
                dat['web-scrapper-order'] = ''
                dat['date_debut'] = date_debut
                dat['date_fin'] = date_fin
                dat['prix_init'] = prix_init
                dat['prix_actuel'] = prix_actuel
                dat['typologie'] = typologie
                dat['n_offre'] = n_offres
                dat['nom'] = residence
                dat['localite'] = localisation
                dat['date_debut-jour'] = ''
                dat['Nb semaines'] = datetime.strptime(
                    date_debut, '%d/%m/%Y').isocalendar()[1]
                dat['url'] = driver.current_url

                data.append(dat)
                print(data)


In [26]:
extract()

[{'web-scrapper-order': '', 'date_debut': '29/06/2024', 'date_fin': '05/07/2024', 'prix_init': '332', 'prix_actuel': 0.0, 'typologie': 'Studio 3 personnes', 'n_offre': '45975', 'nom': 'Résidence Pierre & Vacances Les Combes', 'localite': 'Les Ménuires', 'date_debut-jour': '', 'Nb semaines': 26, 'url': 'https://www.maeva.com/fr-fr/residence-pierre---vacances-les-combes_10790.html'}]
[{'web-scrapper-order': '', 'date_debut': '29/06/2024', 'date_fin': '05/07/2024', 'prix_init': '332', 'prix_actuel': 0.0, 'typologie': 'Studio 3 personnes', 'n_offre': '45975', 'nom': 'Résidence Pierre & Vacances Les Combes', 'localite': 'Les Ménuires', 'date_debut-jour': '', 'Nb semaines': 26, 'url': 'https://www.maeva.com/fr-fr/residence-pierre---vacances-les-combes_10790.html'}, {'web-scrapper-order': '', 'date_debut': '30/03/2024', 'date_fin': '05/04/2024', 'prix_init': '677', 'prix_actuel': 0.0, 'typologie': 'Appartement 5 personnes - 1 chambre', 'n_offre': '45976', 'nom': 'Résidence Pierre & Vacances L